In [1]:
from bokeh.layouts import column
from bokeh.models import CustomJS, ColumnDataSource, Slider, DatetimeTickFormatter, Div
from bokeh.io import output_file, show
import pandas as pd
import numpy as np

from bokeh.plotting import figure

import matplotlib.pyplot as plt
%matplotlib widget
plt.rcParams['figure.figsize'] = (14,10)
plt.rcParams["image.cmap"] = "tab10"
plt.rcParams['axes.prop_cycle'] = plt.cycler(color=plt.cm.tab10.colors)
#Settings for plotting
fs_label = 16
parameters = {
                'axes.labelsize': fs_label,
                'axes.titlesize': fs_label+4,
                'xtick.labelsize': fs_label,
                'ytick.labelsize': fs_label, 
                'legend.fontsize': fs_label, 
                'lines.markersize': 10,
                'lines.linewidth': 3
             }
plt.rcParams.update(parameters)

%matplotlib widget
import matplotlib.colors as colors
from matplotlib import cm # Colormaps

def cmaptohex(tup):
    (r,g,b,a) = tup
    return "#{:02x}{:02x}{:02x}".format(int(r*255),int(g*255),int(b*255))
    
import locale
import matplotlib.dates as mdates
locale.setlocale(locale.LC_TIME,"Danish")
locale.setlocale(locale.LC_ALL,"Danish")

# cmap = plt.cm.get_cmap('Dark2',len(ageGroups))

'Danish_Denmark.1252'

# Load data 

In [2]:
ssidatapath = "ssi_data"
rootdir = os.getcwd()[:-5] + ssidatapath
dfAdm = pd.read_excel('..\Admitted\Admitted.xlsx')
dfAdm = dfAdm.transpose()
dfAdm.columns = dfAdm.iloc[0]
dfAdm = dfAdm.drop(['Aldersgruppe']) 
curDates =  pd.to_datetime(dfAdm.index,format='%d_%m_%Y')
dfCase = pd.DataFrame(columns=dfAdm.columns)

In [3]:
for k in range(0,len(curDates)):
    d = curDates[k]
    curIndex = dfAdm.index[k]

    thisDateStr = d.strftime('%Y-%m-%d')
    curDir = rootdir + '/SSI_data_' + thisDateStr 
    curFilePath = curDir + '/Cases_by_age.csv'
    casedatadf = pd.read_csv(curFilePath,delimiter=';',dtype=str)

    casedatadf['Antal_bekræftede_COVID-19'] = pd.to_numeric(casedatadf['Antal_bekræftede_COVID-19'].astype(str).apply(lambda x: x.replace('.','')))

    dfCase.loc[curIndex] = casedatadf['Antal_bekræftede_COVID-19'].values

# Data: Collect the data in same style

In [4]:
# Same things, but with cases
dfCaseDiff = dfCase.diff().iloc[1:]

dfCaseDiff['Dato'] = curDates[1:]

# Bad hack to ignore 10-15 year olds!
dfCaseDiff['10-19'] = np.multiply(dfCaseDiff['10-19'],0.4)

dfCaseDiffSum = dfCaseDiff.iloc[:,1:-2].cumsum(axis=1)
dfCaseDiffSum['Dato'] = curDates[1:]

# dfCaseDiff = dfCaseDiff.iloc[:-1]
# dfCaseDiffSum = dfCaseDiffSum.iloc[:-1]


dfCaseDiffSum['I alt'] = dfCaseDiffSum['90+']
# dfCaseDiffSum

In [5]:
dfAdmDiff = dfAdm.diff().iloc[1:]

dfAdmDiff['Dato'] = curDates[1:]

dfAdmDiffSum = dfAdmDiff.iloc[:,1:-2].cumsum(axis=1)
dfAdmDiffSum['Dato'] = curDates[1:]

# dfAdmDiff = dfAdmDiff.iloc[:-1]
# dfAdmDiffSum = dfAdmDiffSum.iloc[:-1]


dfAdmDiffSum['I alt'] = dfAdmDiffSum['90+']

# dfAdmDiffSum

# admdfData = pd.DataFrame()
# admdfData['Dato'] = curDates[1:]
# admdfData['Total'] = np.diff(dfAdm['I alt'].astype(int).values)
# admdfData = admdfData.iloc[:-1]
# admdfData 

# Load calculations

In [6]:
datafilename = '../model_results_spread_prediction_var_vacc.csv'
df = pd.read_csv(datafilename)
df.End =  pd.to_datetime(df.End,format='%Y-%m-%d')

# Add the RT2 columns
df['RT2'] = np.round(np.multiply(df.RT,10)).astype(int)

# Only take the vaccination-effeciency of 0.75
curdf = df[df.vacc1 == 0.75]

In [7]:
# Also load the delayed one
datafilenameD = '../model_results_spread_prediction_var_vacc_delayed.csv'
dfD = pd.read_csv(datafilenameD)
dfD.End =  pd.to_datetime(dfD.End,format='%Y-%m-%d')

# Add the RT2 columns
dfD['RT2'] = np.round(np.multiply(dfD.RT,10)).astype(int)

# Only take the vaccination-effeciency of 0.75
curdfD = dfD[dfD.vacc1 == 0.75]

In [8]:
# curdf[curdf.Age == '30-39'] #.End.iloc[-10:]

# Nye smittetilfælde

In [9]:

# curdf['x'] = np.arange(0,len(curdf))
# curdf['y'] = np.arange(0,len(curdf))

casedf = pd.DataFrame()

casedf['Date'] = curdf.End.unique().astype('datetime64[D]')
numVals = len(curdf.End.unique())
firstWeek = 20
weekNums = np.arange(firstWeek,firstWeek+numVals)
casedf['Week'] = weekNums

 


allAges = df.Age.unique()

allRTs = df.RT2.unique()


casedf['Total'] = np.zeros(curdf[(curdf.Age == '20-29') & (curdf.RT2 == 12)].Count.values.shape)
prevVals = np.zeros(curdf[(curdf.Age == '20-29') & (curdf.RT2 == 12)].Count.values.shape)

for curRT in allRTs:
    curColName = 'Total_'+str(curRT)
    casedf[curColName] = np.zeros(curdf[(curdf.Age == '20-29') & (curdf.RT2 == 12)].Count.values.shape)



for curAge in allAges:
    newVals = curdf[(curdf.Age == curAge) & (curdf.RT2 == 12)].Count.values 
    curVals = prevVals + newVals
    # casedf[curAge] = curdf[(curdf.Age == curAge) & (curdf.RT2 == 12)].Count.values 
    casedf[curAge] = curVals
    prevVals = curVals

    casedf['Total'] += curdf[(curdf.Age == curAge) & (curdf.RT2 == 12)].Count.values 

    for curRT in allRTs:
        curColName = curAge+'_'+str(curRT)
        casedf[curColName] = curdf[(curdf.Age == curAge) & (curdf.RT2 == curRT)].Count.values 
        casedf['Total_'+str(curRT)] += curdf[(curdf.Age == curAge) & (curdf.RT2 == curRT)].Count.values 


# # Remove everythings after september first
# casedf = casedf[casedf.Date <= np.datetime64('2021-09-01')]

# casedf

In [18]:
curSumTotal = int(np.round(casedf.Total_12.sum())) # All weeks
curSumTotal = int(np.round(casedf.Total_12[1:].sum())) # Excluding the current weeks (i.e. data)
# casedf
curSumTotal

81463

In [27]:

# x = [x*0.005 for x in range(0, 201)] 

# source = ColumnDataSource(data=dict(x=x, y=x))
# source = ColumnDataSource(data=dict(x=curdf.End, y=curdf.Count, RT09=curdf[curdf.RT == 0.9].Count))
source = ColumnDataSource(casedf)

figW = 1000
figH = 600

curMaxY = 1.1*max(casedf.Total_17)
# curMaxY = 230000
# firstDate = np.datetime64('2021-05-18')
firstDate = np.datetime64('2021-05-25')
predDayTest = '25. maj'
lastDate = np.datetime64('2021-09-01')
plot = figure(plot_width=figW, plot_height=figH, x_axis_type="datetime",x_range = (firstDate,lastDate),tools=[])
plot.y_range.start = 0

plot.toolbar.logo = None
plot.yaxis.axis_label = 'Nye smittetilfælde om ugen'

# x_values_timestamp = [int(el.timestamp()) for el in casedf['Date']]
# x_values_labels = [el.strftime('%d. %B') for el in casedf['Date']]

plot.xaxis.formatter=DatetimeTickFormatter(days="%d. %b",
months="%d. %b",
hours="%d. %b",
minutes="%d. %b")

# plot.xaxis.ticker = x_values_timestamp
# plot.xaxis.major_label_overrides = dict(casedf['Date'].values,x_values_labels))


# plot.y_range.bounds = (0,None)
cmap = plt.cm.get_cmap('terrain',len(allAges)+1)


curAge = allAges[0]
curColor = cmaptohex(cmap(0))
plot.varea('Date', 0 ,'10-19', source=source, fill_color=curColor,legend_label='16-19')
# plot.varea('Date', 0 ,'10-19', source=source, fill_color=curColor,legend_label=curAge)
# plot.line('Week', '10-19', source=source, line_width=3, line_alpha=0.6, color=curColor)

for k in range(1,len(allAges)):
    curAge = allAges[k]
    prevAge = allAges[k-1]
    curColor = cmaptohex(cmap(k))
    # plot.line('Week', curAge, source=source, line_width=3, line_alpha=0.6, color=curColor)
    
    plot.varea('Date', prevAge, curAge, source=source, fill_color=curColor,legend_label=curAge)


plot.line('Date', 'Total', source=source, line_width=3, line_alpha=0.6,color='black',legend_label='Total')


# testDates = np.arange(np.datetime64('2021-05-01'),np.datetime64('2021-05-30'))
# testData = np.multiply(np.ones((len(testDates),)),5000)
# plot.line(testDates,testData)

minRT = 0.9
maxRT = 1.7
slider = Slider(start=minRT, end=maxRT, value=1.2, step=.1, title="Effektivt kontakttal (Rt)", width = figW - 200,align='center',bar_color='#606060')

divBelow = Div(text="Samlet for "+predDayTest+" og frem: "+str(curSumTotal),width=200, height=100)

update_curve = CustomJS(args=dict(source=source, slider=slider, divBelow=divBelow,predDayTest=predDayTest), code="""
    var data = source.data;
    var f = slider.value;
    var curRT = Math.round(f*10)
    var y1 = data['10-19']
    var y2 = data['20-29']
    var y3 = data['30-39']
    var y4 = data['40-49']
    var y5 = data['50-59']
    var y6 = data['60-69']
    var y7 = data['70-79']
    var y8 = data['80-89']
    var y9 = data['90+']
    var yT = data['Total']

    const curName1 = '10-19_'+curRT;
    const curName2 = '20-29_'+curRT;
    const curName3 = '30-39_'+curRT;
    const curName4 = '40-49_'+curRT;
    const curName5 = '50-59_'+curRT;
    const curName6 = '60-69_'+curRT;
    const curName7 = '70-79_'+curRT;
    const curName8 = '80-89_'+curRT;
    const curName9 = '90+_'+curRT; 
    const curNameT = 'Total_'+curRT; 

    // Summing up
    for (var i = 0; i < y1.length; i++) {
        y1[i] = data[curName1][i]
        y2[i] = data[curName2][i] + y1[i]
        y3[i] = data[curName3][i] + y2[i]
        y4[i] = data[curName4][i] + y3[i]
        y5[i] = data[curName5][i] + y4[i]
        y6[i] = data[curName6][i] + y5[i]
        y7[i] = data[curName7][i] + y6[i]
        y8[i] = data[curName8][i] + y7[i]
        y9[i] = data[curName9][i] + y8[i]
        yT[i] = data[curNameT][i]
    }

    var totSum = yT.slice(1).reduce((a, b) => a + b)
    divBelow.text = 'Samlet for '+predDayTest+' og frem: '+String(Math.round(totSum));
    
    // necessary becasue we mutated source.data in-place
    source.change.emit();
""")
slider.js_on_change('value', update_curve)
slider.js_on_change('start', update_curve)

print(plot.y_range.bounds)
 
output_file("cases.html",title='Nye smittetilfælde')

show(column(plot,slider,divBelow))

None


In [28]:

# x = [x*0.005 for x in range(0, 201)] 

# source = ColumnDataSource(data=dict(x=x, y=x))
# source = ColumnDataSource(data=dict(x=curdf.End, y=curdf.Count, RT09=curdf[curdf.RT == 0.9].Count))
source = ColumnDataSource(casedf)
sourceData = ColumnDataSource(dfCaseDiffSum)

figW = 1000
figH = 600

curMaxY = 1.1*max(casedf.Total_17)
# curMaxY = 130000
firstDate = np.datetime64('2021-05-18')
predDayTest = '25. maj'
firstDate = np.datetime64('2021-04-01')
lastDate = np.datetime64('2021-09-01')
plot = figure(plot_width=figW, plot_height=figH, x_axis_type="datetime",x_range = (firstDate,lastDate),tools=[])
# plot = figure(plot_width=figW, plot_height=figH, x_axis_type="datetime",x_range = (firstDate,lastDate),y_range = (0,None),tools=[])
plot.y_range.start = 0

plot.toolbar.logo = None
plot.yaxis.axis_label = 'Nye smittetilfælde om ugen'

# x_values_timestamp = [int(el.timestamp()) for el in casedf['Date']]
# x_values_labels = [el.strftime('%d. %B') for el in casedf['Date']]

plot.xaxis.formatter=DatetimeTickFormatter(days="%d. %b",
months="%d. %b",
hours="%d. %b",
minutes="%d. %b")

# plot.xaxis.ticker = x_values_timestamp
# plot.xaxis.major_label_overrides = dict(casedf['Date'].values,x_values_labels))


# plot.y_range.bounds = (0,None)
cmap = plt.cm.get_cmap('terrain',len(allAges)+1)


curAge = allAges[0]
curColor = cmaptohex(cmap(0))
plot.varea('Date', 0 ,'10-19', source=source, fill_color=curColor,legend_label='16-19')
# plot.varea('Date', 0 ,'10-19', source=source, fill_color=curColor,legend_label=curAge)
# plot.line('Week', '10-19', source=source, line_width=3, line_alpha=0.6, color=curColor)

for k in range(1,len(allAges)):
    curAge = allAges[k]
    prevAge = allAges[k-1]
    curColor = cmaptohex(cmap(k))
    # plot.line('Week', curAge, source=source, line_width=3, line_alpha=0.6, color=curColor)
    
    plot.varea('Date', prevAge, curAge, source=source, fill_color=curColor,legend_label=curAge)


plot.line('Date', 'Total', source=source, line_width=3, line_alpha=0.6,color='black',legend_label='Total')



for k in range(0,len(allAges)):
    curAge = allAges[k]
    curColor = cmaptohex(cmap(k))
    plot.line('Dato', curAge, source=sourceData,color=curColor)
    plot.circle('Dato', curAge, source=sourceData,color=curColor)

plot.circle('Dato', 'I alt', source=sourceData,color='black',legend_label='Data')
plot.line('Dato', 'I alt', source=sourceData, line_width=3, line_alpha=0.6,color='black')


# testDates = np.arange(np.datetime64('2021-05-01'),np.datetime64('2021-05-30'))
# testData = np.multiply(np.ones((len(testDates),)),5000)
# plot.line(testDates,testData)

minRT = 0.9
maxRT = 1.7
slider = Slider(start=minRT, end=maxRT, value=1.2, step=.1, title="Effektivt kontakttal (Rt)", width = figW - 200,align='center',bar_color='#606060')

divBelow = Div(text="Samlet for "+predDayTest+" og frem: "+str(curSumTotal),width=200, height=100)
# divBelow = Div(text="Samlet over hele perioden: "+str(curSumTotal),width=200, height=100)

update_curve = CustomJS(args=dict(source=source, slider=slider, divBelow=divBelow,predDayTest=predDayTest), code="""
    var data = source.data;
    var f = slider.value;
    var curRT = Math.round(f*10)
    var y1 = data['10-19']
    var y2 = data['20-29']
    var y3 = data['30-39']
    var y4 = data['40-49']
    var y5 = data['50-59']
    var y6 = data['60-69']
    var y7 = data['70-79']
    var y8 = data['80-89']
    var y9 = data['90+']
    var yT = data['Total']

    const curName1 = '10-19_'+curRT;
    const curName2 = '20-29_'+curRT;
    const curName3 = '30-39_'+curRT;
    const curName4 = '40-49_'+curRT;
    const curName5 = '50-59_'+curRT;
    const curName6 = '60-69_'+curRT;
    const curName7 = '70-79_'+curRT;
    const curName8 = '80-89_'+curRT;
    const curName9 = '90+_'+curRT; 
    const curNameT = 'Total_'+curRT; 

    // Summing up
    for (var i = 0; i < y1.length; i++) {
        y1[i] = data[curName1][i]
        y2[i] = data[curName2][i] + y1[i]
        y3[i] = data[curName3][i] + y2[i]
        y4[i] = data[curName4][i] + y3[i]
        y5[i] = data[curName5][i] + y4[i]
        y6[i] = data[curName6][i] + y5[i]
        y7[i] = data[curName7][i] + y6[i]
        y8[i] = data[curName8][i] + y7[i]
        y9[i] = data[curName9][i] + y8[i]
        yT[i] = data[curNameT][i]
    }
    

    var totSum = yT.slice(1).reduce((a, b) => a + b)
    divBelow.text = 'Samlet for '+predDayTest+' og frem: '+String(Math.round(totSum));
    
    
    // necessary becasue we mutated source.data in-place
    source.change.emit();
""")
slider.js_on_change('value', update_curve)
slider.js_on_change('start', update_curve)

print(plot.y_range.bounds)
 
output_file("casesData.html",title='Nye smittetilfælde')

show(column(plot,slider,divBelow))

None


# Hospitaliseringer

In [13]:
curdf['Hosp'] = np.multiply(curdf.Count,curdf.CaseHospRate)
# curdf['Hosp'] = np.multiply(np.multiply(curdf.Count,curdf.CaseHospRate),0.75)
# curdf[curdf.Age == 'l0-19'].CaseHospRate.iloc[0]

<ipython-input-13-46103b8eeb16>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  curdf['Hosp'] = np.multiply(curdf.Count,curdf.CaseHospRate)


In [14]:

# curdf['x'] = np.arange(0,len(curdf))
# curdf['y'] = np.arange(0,len(curdf))

admdf = pd.DataFrame()

admdf['Date'] = curdf.End.unique().astype('datetime64[D]')
numVals = len(curdf.End.unique())
firstWeek = 20
weekNums = np.arange(firstWeek,firstWeek+numVals)
admdf['Week'] = weekNums

 


allAges = df.Age.unique()

allRTs = df.RT2.unique()


admdf['Total'] = np.zeros(curdf[(curdf.Age == '20-29') & (curdf.RT2 == 12)].Hosp.values.shape)
prevVals = np.zeros(curdf[(curdf.Age == '20-29') & (curdf.RT2 == 12)].Hosp.values.shape)

for curRT in allRTs:
    curColName = 'Total_'+str(curRT)
    admdf[curColName] = np.zeros(curdf[(curdf.Age == '20-29') & (curdf.RT2 == 12)].Hosp.values.shape)



for curAge in allAges:
    newVals = curdf[(curdf.Age == curAge) & (curdf.RT2 == 12)].Hosp.values 
    curVals = prevVals + newVals
    # admdf[curAge] = curdf[(curdf.Age == curAge) & (curdf.RT2 == 12)].Hosp.values 
    admdf[curAge] = curVals
    prevVals = curVals

    admdf['Total'] += curdf[(curdf.Age == curAge) & (curdf.RT2 == 12)].Hosp.values 

    for curRT in allRTs:
        curColName = curAge+'_'+str(curRT)
        admdf[curColName] = curdf[(curdf.Age == curAge) & (curdf.RT2 == curRT)].Hosp.values 
        admdf['Total_'+str(curRT)] += curdf[(curdf.Age == curAge) & (curdf.RT2 == curRT)].Hosp.values 


# # Remove everythings after september first
# admdf = admdf[admdf.Date <= np.datetime64('2021-09-01')]

# admdf

In [15]:
curSumAdm = int(np.round(admdf.Total_12.sum()))
curSumAdm

2342

In [16]:
# dfAdm = pd.read_excel('..\Admitted\Admitted.xlsx')
# dfAdm = dfAdm.transpose()
# dfAdm.columns = dfAdm.iloc[0]
# dfAdm = dfAdm.drop(['Aldersgruppe']) 
# curDates =  pd.to_datetime(dfAdm.index,format='%d_%m_%Y')


# dfAdmDiff = dfAdm.diff().iloc[1:]

# dfAdmDiff['Dato'] = curDates[1:]

# dfAdmDiffSum = dfAdmDiff.iloc[:,1:-2].cumsum(axis=1)
# dfAdmDiffSum['Dato'] = curDates[1:]

# dfAdmDiff = dfAdmDiff.iloc[:-1]
# dfAdmDiffSum = dfAdmDiffSum.iloc[:-1]


# dfAdmDiffSum['I alt'] = dfAdmDiffSum['90+']
# dfAdmDiffSum

# admdfData = pd.DataFrame()
# admdfData['Dato'] = curDates[1:]
# admdfData['Total'] = np.diff(dfAdm['I alt'].astype(int).values)
# admdfData = admdfData.iloc[:-1]
# admdfData 

In [29]:

from bokeh.models import DatetimeTickFormatter
# x = [x*0.005 for x in range(0, 201)] 

# source = ColumnDataSource(data=dict(x=x, y=x))
# source = ColumnDataSource(data=dict(x=curdf.End, y=curdf.Count, RT09=curdf[curdf.RT == 0.9].Count))
source = ColumnDataSource(admdf)
sourceData = ColumnDataSource(dfAdmDiffSum)

figW = 1000
figH = 600

curMaxY = 1.1*max(admdf.Total_17)
# curMaxY = 30000
# curMaxY = 600

firstDate = np.datetime64('2021-05-18')
firstDate = np.datetime64('2021-05-25')
predDayTest = '25. maj'
lastDate = np.datetime64('2021-09-01')
plot = figure(plot_width=figW, plot_height=figH, x_axis_type="datetime",x_range = (firstDate,lastDate),tools=[])
plot.y_range.start = 0

# firstDate = np.datetime64('2021-04-01')
# lastDate = np.datetime64('2021-09-01')
# plot = figure(plot_width=figW, plot_height=figH, x_axis_type="datetime",x_range = (firstDate,lastDate),y_range = (0,curMaxY),tools=[])

plot.toolbar.logo = None
plot.yaxis.axis_label = 'Nye hospitalindlæggelser om ugen'

# x_values_timestamp = [int(el.timestamp()) for el in casedf['Date']]
# x_values_labels = [el.strftime('%d. %B') for el in casedf['Date']]

plot.xaxis.formatter=DatetimeTickFormatter(days="%d. %b",
months="%d. %b",
hours="%d. %b",
minutes="%d. %b")

# plot.xaxis.ticker = x_values_timestamp
# plot.xaxis.major_label_overrides = dict(casedf['Date'].values,x_values_labels))


# plot.y_range.bounds = (0,None)
cmap = plt.cm.get_cmap('terrain',len(allAges)+1)


curAge = allAges[0]
curColor = cmaptohex(cmap(0))
plot.varea('Date', 0 ,'10-19', source=source, fill_color=curColor,legend_label='16-19')
# plot.varea('Date', 0 ,'10-19', source=source, fill_color=curColor,legend_label=curAge)
# plot.line('Week', '10-19', source=source, line_width=3, line_alpha=0.6, color=curColor)

for k in range(1,len(allAges)):
    curAge = allAges[k]
    prevAge = allAges[k-1]
    curColor = cmaptohex(cmap(k))
    # plot.line('Week', curAge, source=source, line_width=3, line_alpha=0.6, color=curColor)
    
    plot.varea('Date', prevAge, curAge, source=source, fill_color=curColor,legend_label=curAge)


plot.line('Date', 'Total', source=source, line_width=3, line_alpha=0.6,color='black',legend_label='Total')



# for k in range(0,len(allAges)):
#     curAge = allAges[k]
#     curColor = cmaptohex(cmap(k))
#     plot.line('Dato', curAge, source=sourceData,color=curColor)
#     plot.circle('Dato', curAge, source=sourceData,color=curColor)

# plot.circle('Dato', 'I alt', source=sourceData,color='black',legend_label='Data')
# plot.line('Dato', 'I alt', source=sourceData, line_width=3, line_alpha=0.6,color='black')


minRT = 0.9
maxRT = 1.7
slider = Slider(start=minRT, end=maxRT, value=1.2, step=.1, title="Effektivt kontakttal (Rt)", width = figW - 200,align='center',bar_color='#606060')

divBelow = Div(text="Samlet for "+predDayTest+" og frem: "+str(curSumAdm),width=200, height=100)
# divBelow = Div(text="Samlet over hele perioden: "+str(curSumAdm),width=200, height=100)

update_curve = CustomJS(args=dict(source=source, slider=slider, divBelow=divBelow,predDayTest=predDayTest), code="""
    var data = source.data;
    var f = slider.value;
    var curRT = Math.round(f*10)
    var y1 = data['10-19']
    var y2 = data['20-29']
    var y3 = data['30-39']
    var y4 = data['40-49']
    var y5 = data['50-59']
    var y6 = data['60-69']
    var y7 = data['70-79']
    var y8 = data['80-89']
    var y9 = data['90+']
    var yT = data['Total']

    const curName1 = '10-19_'+curRT;
    const curName2 = '20-29_'+curRT;
    const curName3 = '30-39_'+curRT;
    const curName4 = '40-49_'+curRT;
    const curName5 = '50-59_'+curRT;
    const curName6 = '60-69_'+curRT;
    const curName7 = '70-79_'+curRT;
    const curName8 = '80-89_'+curRT;
    const curName9 = '90+_'+curRT; 
    const curNameT = 'Total_'+curRT; 

    // Summing up
    for (var i = 0; i < y1.length; i++) {
        y1[i] = data[curName1][i]
        y2[i] = data[curName2][i] + y1[i]
        y3[i] = data[curName3][i] + y2[i]
        y4[i] = data[curName4][i] + y3[i]
        y5[i] = data[curName5][i] + y4[i]
        y6[i] = data[curName6][i] + y5[i]
        y7[i] = data[curName7][i] + y6[i]
        y8[i] = data[curName8][i] + y7[i]
        y9[i] = data[curName9][i] + y8[i]
        yT[i] = data[curNameT][i]
    }
    

    var totSum = yT.slice(1).reduce((a, b) => a + b)
    divBelow.text = 'Samlet for '+predDayTest+' og frem: '+String(Math.round(totSum));
    
    
    // necessary becasue we mutated source.data in-place
    source.change.emit();
""")
slider.js_on_change('value', update_curve)
slider.js_on_change('start', update_curve)

print(plot.y_range.bounds)
 
output_file("hosp.html",title='Hospitaliseringer')

show(column(plot,slider,divBelow))

NameError: name 'admdf' is not defined

In [18]:

from bokeh.models import DataRange1d
from bokeh.models import DatetimeTickFormatter
# x = [x*0.005 for x in range(0, 201)] 

# source = ColumnDataSource(data=dict(x=x, y=x))
# source = ColumnDataSource(data=dict(x=curdf.End, y=curdf.Count, RT09=curdf[curdf.RT == 0.9].Count))
source = ColumnDataSource(admdf)
sourceData = ColumnDataSource(dfAdmDiffSum)

figW = 1000
figH = 600

# curMaxY = 1.1*max(admdf.Total_17)
# curMaxY = 600


# firstDate = np.datetime64('2021-05-18')
# lastDate = np.datetime64('2021-09-01')
# plot = figure(plot_width=figW, plot_height=figH, x_axis_type="datetime",x_range = (firstDate,lastDate),y_range = (0,curMaxY),tools=[])

firstDate = np.datetime64('2021-04-01')
predDayTest = '25. maj'
lastDate = np.datetime64('2021-09-01')
plot = figure(plot_width=figW, plot_height=figH, x_axis_type="datetime",x_range = (firstDate,lastDate),tools=[])
plot.y_range.start = 0

plot.toolbar.logo = None
plot.yaxis.axis_label = 'Nye hospitalindlæggelser om ugen'

# x_values_timestamp = [int(el.timestamp()) for el in casedf['Date']]
# x_values_labels = [el.strftime('%d. %B') for el in casedf['Date']]

plot.xaxis.formatter=DatetimeTickFormatter(days="%d. %b",
months="%d. %b",
hours="%d. %b",
minutes="%d. %b")

# plot.xaxis.ticker = x_values_timestamp
# plot.xaxis.major_label_overrides = dict(casedf['Date'].values,x_values_labels))


# plot.y_range.bounds = (0,None)
cmap = plt.cm.get_cmap('terrain',len(allAges)+1)


curAge = allAges[0]
curColor = cmaptohex(cmap(0))
plot.varea('Date', 0 ,'10-19', source=source, fill_color=curColor,legend_label='16-19')
# plot.varea('Date', 0 ,'10-19', source=source, fill_color=curColor,legend_label=curAge)
# plot.line('Week', '10-19', source=source, line_width=3, line_alpha=0.6, color=curColor)

for k in range(1,len(allAges)):
    curAge = allAges[k]
    prevAge = allAges[k-1]
    curColor = cmaptohex(cmap(k))
    # plot.line('Week', curAge, source=source, line_width=3, line_alpha=0.6, color=curColor)
    
    plot.varea('Date', prevAge, curAge, source=source, fill_color=curColor,legend_label=curAge)


plot.line('Date', 'Total', source=source, line_width=3, line_alpha=0.6,color='black',legend_label='Total')



for k in range(0,len(allAges)):
    curAge = allAges[k]
    curColor = cmaptohex(cmap(k))
    plot.line('Dato', curAge, source=sourceData,color=curColor)
    plot.circle('Dato', curAge, source=sourceData,color=curColor)

plot.circle('Dato', 'I alt', source=sourceData,color='black',legend_label='Data')
plot.line('Dato', 'I alt', source=sourceData, line_width=3, line_alpha=0.6,color='black')


minRT = 0.9
maxRT = 1.7
slider = Slider(start=minRT, end=maxRT, value=1.2, step=.1, title="Effektivt kontakttal (Rt)", width = figW - 200,align='center',bar_color='#606060')

divBelow = Div(text="Samlet for "+predDayTest+" og frem: "+str(curSumAdm),width=200, height=100)
# divBelow = Div(text="Samlet over hele perioden: "+str(curSumAdm),width=200, height=100)

update_curve = CustomJS(args=dict(source=source, slider=slider, divBelow=divBelow,predDayTest=predDayTest), code="""
    var data = source.data;
    var f = slider.value;
    var curRT = Math.round(f*10)
    var y1 = data['10-19']
    var y2 = data['20-29']
    var y3 = data['30-39']
    var y4 = data['40-49']
    var y5 = data['50-59']
    var y6 = data['60-69']
    var y7 = data['70-79']
    var y8 = data['80-89']
    var y9 = data['90+']
    var yT = data['Total']

    const curName1 = '10-19_'+curRT;
    const curName2 = '20-29_'+curRT;
    const curName3 = '30-39_'+curRT;
    const curName4 = '40-49_'+curRT;
    const curName5 = '50-59_'+curRT;
    const curName6 = '60-69_'+curRT;
    const curName7 = '70-79_'+curRT;
    const curName8 = '80-89_'+curRT;
    const curName9 = '90+_'+curRT; 
    const curNameT = 'Total_'+curRT; 

    // Summing up
    for (var i = 0; i < y1.length; i++) {
        y1[i] = data[curName1][i]
        y2[i] = data[curName2][i] + y1[i]
        y3[i] = data[curName3][i] + y2[i]
        y4[i] = data[curName4][i] + y3[i]
        y5[i] = data[curName5][i] + y4[i]
        y6[i] = data[curName6][i] + y5[i]
        y7[i] = data[curName7][i] + y6[i]
        y8[i] = data[curName8][i] + y7[i]
        y9[i] = data[curName9][i] + y8[i]
        yT[i] = data[curNameT][i]
    }
    
    
    var totSum = yT.slice(1).reduce((a, b) => a + b)
    divBelow.text = 'Samlet for '+predDayTest+' og frem: '+String(Math.round(totSum));
    
    
    // necessary becasue we mutated source.data in-place
    source.change.emit();
""")
slider.js_on_change('value', update_curve)
slider.js_on_change('start', update_curve)

print(plot.y_range.bounds)
 
output_file("hospData.html",title='Hospitaliseringer')

show(column(plot,slider,divBelow))

None


# Nye smittetilfælde, delayed

In [19]:

# curdfD['x'] = np.arange(0,len(curdf))
# curdf['y'] = np.arange(0,len(curdf))

casedf = pd.DataFrame()

casedf['Date'] = curdfD.End.unique().astype('datetime64[D]')
numVals = len(curdfD.End.unique())
firstWeek = 20
weekNums = np.arange(firstWeek,firstWeek+numVals)
casedf['Week'] = weekNums

 


allAges = dfD.Age.unique()

allRTs = dfD.RT2.unique()


casedf['Total'] = np.zeros(curdfD[(curdfD.Age == '20-29') & (curdfD.RT2 == 12)].Count.values.shape)
prevVals = np.zeros(curdfD[(curdfD.Age == '20-29') & (curdfD.RT2 == 12)].Count.values.shape)

for curRT in allRTs:
    curColName = 'Total_'+str(curRT)
    casedf[curColName] = np.zeros(curdfD[(curdfD.Age == '20-29') & (curdfD.RT2 == 12)].Count.values.shape)



for curAge in allAges:
    newVals = curdfD[(curdfD.Age == curAge) & (curdfD.RT2 == 12)].Count.values 
    curVals = prevVals + newVals
    # casedf[curAge] = curdfD[(curdfD.Age == curAge) & (curdfD.RT2 == 12)].Count.values 
    casedf[curAge] = curVals
    prevVals = curVals

    casedf['Total'] += curdfD[(curdfD.Age == curAge) & (curdfD.RT2 == 12)].Count.values 

    for curRT in allRTs:
        curColName = curAge+'_'+str(curRT)
        casedf[curColName] = curdfD[(curdfD.Age == curAge) & (curdfD.RT2 == curRT)].Count.values 
        casedf['Total_'+str(curRT)] += curdfD[(curdfD.Age == curAge) & (curdfD.RT2 == curRT)].Count.values 


# # Remove everythings after september first
# casedf = casedf[casedf.Date <= np.datetime64('2021-09-01')]

# casedf
curSumTotal = int(np.round(casedf.Total_12.sum()))
curSumTotal


48002

In [41]:
dfCaseDiffSumTest = dfCaseDiffSum

extraRow = dfCaseDiffSumTest.iloc[len(dfCaseDiffSum)-1]
extraRow.Dato = extraRow.Dato + np.timedelta64(7,'D')
dfCaseDiffSumTest = dfCaseDiffSumTest.reset_index(drop=True)
dfCaseDiffSumTest = dfCaseDiffSumTest.append(extraRow)
dfCaseDiffSumTest = dfCaseDiffSumTest.reset_index(drop=True)


extraRow = dfCaseDiffSumTest.iloc[len(dfCaseDiffSumTest)-1]
extraRow.Dato = extraRow.Dato + np.timedelta64(7,'D')
dfCaseDiffSumTest = dfCaseDiffSumTest.reset_index(drop=True)
dfCaseDiffSumTest = dfCaseDiffSumTest.append(extraRow)
dfCaseDiffSumTest = dfCaseDiffSumTest.reset_index(drop=True)

dfCaseDiffSumTest = dfCaseDiffSumTest.iloc[-3:]
# dfCaseDiffSumTest

# dfCaseDiffSumTest.Dato = pd.to_datetime(dfCaseDiffSumTest.Dato)
# dfCaseDiffSumTest.iloc[-1].Dato = dfCaseDiffSumTest.iloc[-2].Dato + np.timedelta64(14,'D')

 
# dfCaseDiffSumTest
# # extraRow
# dfCaseDiffSum.loc[len(dfCaseDiffSum)] = extraRow
# dfCaseDiffSum.iloc[len(dfCaseDiffSum)] = extraRow
# # dfCaseDiffSum.loc[dfCaseDiffSum.index.repeat(2)]
# # dfCaseDiffSum.iloc[-1+1] = extraRow 
# # dfCaseDiffSum
# # dfCaseDiffSum.append(pd.DataFrame(extraRow,columns=dfCaseDiffSum.columns),ignore_index=True)
# # dfCaseDiffSum.append(extraRow,ignore_index=True)

C:\ProgramData\Anaconda3\envs\main\lib\site-packages\pandas\core\generic.py:5494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [42]:

# x = [x*0.005 for x in range(0, 201)] 

# source = ColumnDataSource(data=dict(x=x, y=x))
# source = ColumnDataSource(data=dict(x=curdf.End, y=curdf.Count, RT09=curdf[curdf.RT == 0.9].Count))
source = ColumnDataSource(casedf)
sourceData = ColumnDataSource(dfCaseDiffSum)
sourceDataDel = ColumnDataSource(dfCaseDiffSumTest)

figW = 1000
figH = 600

curMaxY = 1.1*max(casedf.Total_17)
# curMaxY = 130000
firstDate = np.datetime64('2021-05-18')
predDayTest = '25. maj'
firstDate = np.datetime64('2021-04-01')
lastDate = np.datetime64('2021-09-01')
plot = figure(plot_width=figW, plot_height=figH, x_axis_type="datetime",x_range = (firstDate,lastDate),tools=[])
# plot = figure(plot_width=figW, plot_height=figH, x_axis_type="datetime",x_range = (firstDate,lastDate),y_range = (0,None),tools=[])
plot.y_range.start = 0

plot.toolbar.logo = None
plot.yaxis.axis_label = 'Nye smittetilfælde om ugen'

# x_values_timestamp = [int(el.timestamp()) for el in casedf['Date']]
# x_values_labels = [el.strftime('%d. %B') for el in casedf['Date']]

plot.xaxis.formatter=DatetimeTickFormatter(days="%d. %b",
months="%d. %b",
hours="%d. %b",
minutes="%d. %b")

# plot.xaxis.ticker = x_values_timestamp
# plot.xaxis.major_label_overrides = dict(casedf['Date'].values,x_values_labels))


# plot.y_range.bounds = (0,None)
cmap = plt.cm.get_cmap('terrain',len(allAges)+1)


curAge = allAges[0]
curColor = cmaptohex(cmap(0))
plot.varea('Date', 0 ,'10-19', source=source, fill_color=curColor,legend_label='16-19')
# plot.varea('Date', 0 ,'10-19', source=source, fill_color=curColor,legend_label=curAge)
# plot.line('Week', '10-19', source=source, line_width=3, line_alpha=0.6, color=curColor)

for k in range(1,len(allAges)):
    curAge = allAges[k]
    prevAge = allAges[k-1]
    curColor = cmaptohex(cmap(k))
    # plot.line('Week', curAge, source=source, line_width=3, line_alpha=0.6, color=curColor)
    
    plot.varea('Date', prevAge, curAge, source=source, fill_color=curColor,legend_label=curAge)


plot.line('Date', 'Total', source=source, line_width=3, line_alpha=0.6,color='black',legend_label='Total')



for k in range(0,len(allAges)):
    curAge = allAges[k]
    curColor = cmaptohex(cmap(k))
    plot.line('Dato', curAge, source=sourceData,color=curColor)
    plot.circle('Dato', curAge, source=sourceData,color=curColor)

plot.circle('Dato', 'I alt', source=sourceData,color='black',legend_label='Data')
plot.line('Dato', 'I alt', source=sourceData, line_width=3, line_alpha=0.6,color='black')


for k in range(0,len(allAges)):
    curAge = allAges[k]
    curColor = cmaptohex(cmap(k))
    plot.line('Dato', curAge,line_dash="4 4", source=sourceDataDel,color=curColor)
    plot.triangle('Dato', curAge, source=sourceDataDel,color=curColor)

plot.triangle('Dato', 'I alt', source=sourceDataDel,color='black',legend_label='Data')
plot.line('Dato', 'I alt', source=sourceDataDel,line_dash="4 4", line_width=3, line_alpha=0.6,color='black')


# testDates = np.arange(np.datetime64('2021-05-01'),np.datetime64('2021-05-30'))
# testData = np.multiply(np.ones((len(testDates),)),5000)
# plot.line(testDates,testData)

minRT = 0.9
maxRT = 1.7
slider = Slider(start=minRT, end=maxRT, value=1.2, step=.1, title="Effektivt kontakttal (Rt)", width = figW - 200,align='center',bar_color='#606060')

divBelow = Div(text="Samlet for "+predDayTest+" og frem: "+str(curSumTotal),width=200, height=100)
# divBelow = Div(text="Samlet over hele perioden: "+str(curSumTotal),width=200, height=100)

update_curve = CustomJS(args=dict(source=source, slider=slider, divBelow=divBelow,predDayTest=predDayTest), code="""
    var data = source.data;
    var f = slider.value;
    var curRT = Math.round(f*10)
    var y1 = data['10-19']
    var y2 = data['20-29']
    var y3 = data['30-39']
    var y4 = data['40-49']
    var y5 = data['50-59']
    var y6 = data['60-69']
    var y7 = data['70-79']
    var y8 = data['80-89']
    var y9 = data['90+']
    var yT = data['Total']

    const curName1 = '10-19_'+curRT;
    const curName2 = '20-29_'+curRT;
    const curName3 = '30-39_'+curRT;
    const curName4 = '40-49_'+curRT;
    const curName5 = '50-59_'+curRT;
    const curName6 = '60-69_'+curRT;
    const curName7 = '70-79_'+curRT;
    const curName8 = '80-89_'+curRT;
    const curName9 = '90+_'+curRT; 
    const curNameT = 'Total_'+curRT; 

    // Summing up
    for (var i = 0; i < y1.length; i++) {
        y1[i] = data[curName1][i]
        y2[i] = data[curName2][i] + y1[i]
        y3[i] = data[curName3][i] + y2[i]
        y4[i] = data[curName4][i] + y3[i]
        y5[i] = data[curName5][i] + y4[i]
        y6[i] = data[curName6][i] + y5[i]
        y7[i] = data[curName7][i] + y6[i]
        y8[i] = data[curName8][i] + y7[i]
        y9[i] = data[curName9][i] + y8[i]
        yT[i] = data[curNameT][i]
    }
    

    var totSum = yT.reduce((a, b) => a + b)
    divBelow.text = 'Samlet for '+predDayTest+' og frem: '+String(Math.round(totSum));
    
    
    // necessary becasue we mutated source.data in-place
    source.change.emit();
""")
slider.js_on_change('value', update_curve)
slider.js_on_change('start', update_curve)

print(plot.y_range.bounds)
 
output_file("casesDataDelayed.html",title='Nye smittetilfælde, forsinket')

show(column(plot,slider,divBelow))

None


# OLD

In [22]:

# plot.line('Week', '10-19', source=source, line_width=3, line_alpha=0.6, color='#00ff00')
# plot.line('Week', '20-29', source=source, line_width=3, line_alpha=0.6, color='#20dd20')
# plot.line('Week', '30-39', source=source, line_width=3, line_alpha=0.6, color='#307030')
# plot.line('Week', '40-49', source=source, line_width=3, line_alpha=0.6, color='#406040')
# plot.line('Week', '50-59', source=source, line_width=3, line_alpha=0.6, color='#505050')
# plot.line('Week', '60-69', source=source, line_width=3, line_alpha=0.6, color='#604060')
# plot.line('Week', '70-79', source=source, line_width=3, line_alpha=0.6, color='#703070')
# plot.line('Week', '80-89', source=source, line_width=3, line_alpha=0.6, color='#802080')
# plot.line('Week',   '90+', source=source, line_width=3, line_alpha=0.6, color='#901090')
# plot.line('Week', 'Total', source=source, line_width=3, line_alpha=0.6,color='black')

# plot.circle('Week', '10-19', source=source, color='#108000')
# plot.circle('Week', '20-29', source=source, color='#208000')
# plot.circle('Week', '30-39', source=source, color='#308000')
# plot.circle('Week', '40-49', source=source, color='#408000')
# plot.circle('Week', '50-59', source=source, color='#508000')
# plot.circle('Week', '60-69', source=source, color='#608000')
# plot.circle('Week', '70-79', source=source, color='#708000')
# plot.circle('Week', '80-89', source=source, color='#808000')
# plot.circle('Week',   '90+', source=source, color='#908000')
# plot.circle('Week', 'Total', source=source,color='black')
# plot.line('Week', 'Total', source=source, line_width=3, line_alpha=0.6)


# plot.varea('Week', 0 ,'10-19', source=source, fill_color=cmaptohex(cmap(1)))
# plot.varea('Week','10-19', '20-29', source=source, color=cmaptohex(cmap(2)))
# plot.varea('Week','20-29', '30-39', source=source, color=cmaptohex(cmap(3)))
# plot.varea('Week','30-39', '40-49', source=source, color=cmaptohex(cmap(4)))
# plot.varea('Week','40-49', '50-59', source=source, color=cmaptohex(cmap(5)))
# plot.varea('Week','50-59', '60-69', source=source, color=cmaptohex(cmap(6)))
# plot.varea('Week','60-69', '70-79', source=source, color=cmaptohex(cmap(7)))
# plot.varea('Week','70-79', '80-89', source=source, color=cmaptohex(cmap(8)))
# plot.varea('Week','80-89',   '90+', source=source, color=cmaptohex(cmap(9)))

# slider = Slider(start=0.1, end=6, value=1, step=.1, title="power")

In [23]:
# curdf['RT2'] = np.round(np.multiply(curdf.RT,10)).astype(int)


    # for (var i = 0; i < y1.length; i++) {
    #     y1[i] = data[curName1][i]
    #     y2[i] = data[curName2][i]
    #     y3[i] = data[curName3][i]
    #     y4[i] = data[curName4][i]
    #     y5[i] = data[curName5][i]
    #     y6[i] = data[curName6][i]
    #     y7[i] = data[curName7][i]
    #     y8[i] = data[curName8][i]
    #     y9[i] = data[curName9][i]
    #     yT[i] = data[curNameT][i]
    # }


    # // Summing up
    # for (var i = 0; i < y1.length; i++) {
    #     y1[i] = data[curName1][i]
    #     y2[i] = data[curName2][i] + y1[i]
    #     y3[i] = data[curName3][i] + y2[i]
    #     y4[i] = data[curName4][i] + y3[i]
    #     y5[i] = data[curName5][i] + y4[i]
    #     y6[i] = data[curName6][i] + y5[i]
    #     y7[i] = data[curName7][i] + y6[i]
    #     y8[i] = data[curName8][i] + y7[i]
    #     y9[i] = data[curName9][i] + y8[i]
    #     yT[i] = data[curNameT][i]
    # }

In [24]:

# # x = [x*0.005 for x in range(0, 201)] 

# # source = ColumnDataSource(data=dict(x=x, y=x))
# # source = ColumnDataSource(data=dict(x=curdf.End, y=curdf.Count, RT09=curdf[curdf.RT == 0.9].Count))
# source = ColumnDataSource(curdf)

# plot = figure(plot_width=400, plot_height=400)
# plot.line('x', 'Count', source=source, line_width=3, line_alpha=0.6)

# # slider = Slider(start=0.1, end=6, value=1, step=.1, title="power")
# minRT = 0.9
# maxRT = 1.7
# slider = Slider(start=minRT, end=maxRT, value=1.1, step=.1, title="RT")

# update_curve = CustomJS(args=dict(source=source, slider=slider), code="""
#     var data = source.data;
#     var f = slider.value;
#     var age = data['Age']
#     var RT = data['RT2']
#     var End = data['End']
#     var Count = data['Count']

#     var x = data['x']
#     var y = data['y']

#     console.log(Count)
#     for (var i = 0; i < RT.length; i++) {
#         y[i] = 6
#         Count[i] = 0
        
#         if (RT[i] == Math.round(f*10)){
#             y[i] = data['Count'][i]
#             console.log(data['Count'][i])
#             Count[i] = data['Count'][i]

#             // End.push(data['End'][i])
#             // Count.push(data['Count'][i])
#         }

#     }

#     //for (var i = 0; i < x.length; i++) {
#         // y[i] = Math.pow(x[i], f)
#     //}
    
#     // necessary becasue we mutated source.data in-place
#     source.change.emit();
# """)
# slider.js_on_change('value', update_curve)

# output_file("plot.html")

# show(column(slider, plot))

In [25]:

# testdf['Total'] = curdf[(curdf.RT2 == 11)].Count.values 
# for curRT in allRTs:
#     curColName = 'Total_'+str(curRT)
#     testdf[curColName] = curdf[(curdf.RT2 == curRT)].Count.values 

# testdf['20_RT10'] = curdf[(curdf.Age == '20-29') & (curdf.RT2 == 10)].Count.values
# testdf['20_RT11'] = curdf[(curdf.Age == '20-29') & (curdf.RT2 == 11)].Count.values
# testdf['20_RT12'] = curdf[(curdf.Age == '20-29') & (curdf.RT2 == 12)].Count.values
# testdf['20_RT13'] = curdf[(curdf.Age == '20-29') & (curdf.RT2 == 13)].Count.values
# testdf['20_RT14'] = curdf[(curdf.Age == '20-29') & (curdf.RT2 == 14)].Count.values
# testdf['RT11_20'] = curdf[(curdf.Age == '20-29') & (curdf.RT2 == 11)].Count.values
# testdf['RT12_20'] = curdf[(curdf.Age == '20-29') & (curdf.RT2 == 12)].Count.values
# testdf['RT13_20'] = curdf[(curdf.Age == '20-29') & (curdf.RT2 == 13)].Count.values
# testdf['RT14_20'] = curdf[(curdf.Age == '20-29') & (curdf.RT2 == 14)].Count.values

    # if (Math.round(f*10) == 11){
    #     for (var i = 0; i < y.length; i++) {
    #         y[i] = data['RT11_20'][i]
    #     }
    # }
    # if (Math.round(f*10) == 12){
    #     for (var i = 0; i < y.length; i++) {
    #         y[i] = data['RT12_20'][i]
    #     }
    # }
    # if (Math.round(f*10) == 13){
    #     for (var i = 0; i < y.length; i++) {
    #         y[i] = data['RT13_20'][i]
    #     }
    # }
    
